In [1]:
###################################  INSTALL THE NECESSARY LIBRARIES #############################
############## HIGHLIGHT THE LIBRARIES BELOW AND PRESS CRTL + / THEN RUN THE CELL ############### 

# !pip install --upgrade pandas pandas-datareader
# !pip install dash
# !pip install jupyter_dash
# !pip install dash_bootstrap_components
# !pip install daq
# !pip install dash_table
# !pip install dash_daq
# !pip install cython

In [2]:
####################################  IMPORT NECESSARY LIBRARIES #############################

import math as math # Build functions as PI or math.radians(), etc..
import pandas as pd # Data Frame Library
pd.options.mode.chained_assignment = None  # default='warn' # Turn of the Warnings
import matplotlib.pyplot as plt # Graphs Library
import plotly.offline as pyo # DashBoard
import plotly.graph_objects as go # DashBoard
import dash # DashBoard
from dash import html # DashBoard
from dash import dcc # DashBoard
from jupyter_dash import JupyterDash # DashBoard
import dash_bootstrap_components as dbc # DashBoard
from dash.dependencies import Output, Input,State # DashBoard
import dash_daq as daq
from dash import Dash, dash_table, html
import dash.dash_table.FormatTemplate as FormatTemplate
from dash.dash_table.Format import Sign
import pandas as pd
import random
import re #Regular expression operations
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import warnings
warnings.filterwarnings("ignore") #uppress all runtime warnings
import time
from functools import cache
import logging
logging.basicConfig(filename='app.log', level=logging.ERROR)

In [3]:
####################################  MAIN DATAFRAME #################################### 

# Creating data for store the calculation results:
df = pd.DataFrame({'Crank Angle': [*range(-180, 180 + 1, 1)], 'Piston Position [mm]': 0, 
                   'Piston Velocity [mm/deg]': 0, 'Piston Acc [mm/deg/deg]': 0,
                   'Piston Velocity [m/s]': 0, 'Piston Acc [m/s/s]': 0, 'Volume [CC]': 0,
                     'Pressure [Bar]': 0, 'Wiebe': 0, 'Wiebe': 0, 'MFB': 0, 'dQr [J]': 0,
                   'Combustion [bar]': 0,'Temperature [K]': 0, 'Work Done [J]': 0, 
                   'Rate of Change Pressure': 0, 'Rate of Change Combustion': 0, 
                   'Rate of Change Pressure - Combustion': 0
                  })

data = {'Results':[], '-' : [], 'Units': [] }
resultsnoX = pd.DataFrame(data)

dataResults  = {'Engine Cycle Simulation': ['Work Done (Joules)', 'Power (Kw)', 'BHP', 'Capacity (cc)', 'Qr (Joules)'],
                    'Results': [0, 0, 0, 0, 0], } 

df_results = pd.DataFrame(dataResults)

In [4]:
##########################  GLOBAL VARIABLES TO STORE THE INPUT VALUES  ##########################
CARBONMASS = 12
HYDROGENMASS = 1
OXYGENMASS = 16
NITROGENMASS = 14
OxigenPercentage = 21 / 100
NitrogenPercentage = 79 / 100
RatioOxy_Nitrogen = NitrogenPercentage / OxigenPercentage

global CARBONMASS, HYDROGENMASS, OXYGENMASS, NITROGENMASS, RatioOxy_Nitrogen


In [5]:
##########################  FUNCTIONS  ##########################
#Function to generate the table of results from the main function (EngineCalc())
def generate_table(data, max_rows=5):
    return html.Table([
        html.Thead([html.Tr([html.Th(col) for col in data.columns])]),
        html.Tbody([html.Tr([
            html.Td(data.iloc[i][col], style={'padding-top': '5px', 'padding-bottom': '5px', }) for col in data.columns
        ]) for i in range(min(len(data), max_rows))])
    ])


# Main Function to run the engine simualtion
def EngineCalc(n_cylinders, con_rod_length, stroke, bore, compression_ratio, 
               ambient_temperature, throttle, rpm, 
               ignition_delay, ignition_degrees, burn_duration, AFR_FromTable, 
               LHV_FromTable, df):  

    #Engine Cycle Simulation Software

    CompRatio = compression_ratio   # From Manual
    Bore = bore / 1000 # m
    Stroke = stroke / 1000 # m
    Cylinders = n_cylinders
    ConRodLength = con_rod_length / 1000 # m
    RPM = rpm # Rev. per minut
    Capacity = (math.pi * ((0.5 * (Bore))) ** 2) * (Stroke) * 1000000 * Cylinders # cc
    CapacitySI = Capacity / 1000000
    n_compression = 1.28
    n_expansion = 1.2
    Wiebe_a = 5
    Wiebe_m = 2
    Ignition_deg = ignition_degrees
    Ignition_Del = ignition_delay
    Bdur = burn_duration
    if Bdur < (abs(Ignition_deg) + Ignition_Del):
        time.sleep(1)
        raise ValueError("-> Please Increase the burn duration < --")
    atmospheric_pressure = 1 # bar
    R = 286.9 # Universal real gas constant J/kgK
    InitialTemp = ambient_temperature + 273.15 # Kelvin
    NVolume = (throttle / 100) - .1 # Real value if TPS is not assumed = 0.90   Vol. Efficiency
    if throttle < 1 or throttle > 100:
        raise ValueError("-> Please Check the Trothle Value < --")
    CombEfficiency = 0.9 # Comb. Efficiency
    Air_fuel_ratio = AFR_FromTable 
    Calorific_fuel = LHV_FromTable # J/Kg.K            
    DensityAir = 1.2 # kg/m^3
    Gamma = 1.25
    Vts = Cylinders * (math.pi / 4) * ((Bore) ** 2) * (Stroke)  # Total swept volume m^3
    Vsv = Vts / Cylinders  # Swept volume or cylinder volume
    Vcv = Vsv / (CompRatio - 1) # Clearance volume m^3
    mta = Vsv * NVolume * DensityAir  # Mass of gas Kg (Mass Air)
    mtf = mta / Air_fuel_ratio # Mass of fuel trapped Kg (Mass Fuel)
    
    ############ ########## ########## ########## ############ #########
    #Piston
    for i in range(len(df['Crank Angle'])):
        
    # Piston Position: 
        df['Piston Position [mm]'][i]  = (ConRodLength + (0.5 * Stroke) -
                                          ((0.5 * Stroke)* math.cos(math.radians 
                                                                 (df['Crank Angle'][i])) +
                                           ((ConRodLength**2) - ((0.5 * Stroke)**2) *
                                            (math.sin(math.radians(df['Crank Angle'][i])) *
                                             (math.sin(math.radians(df['Crank Angle'][i]))))) **0.5))* 1000

    # Piston Velocity [mm/deg]:  
        if i >= 1:
            df['Piston Velocity [mm/deg]'][i - 1] = df['Piston Position [mm]'][i] - df['Piston Position [mm]'][i - 1]
            
    # Piston Velocity [m/s]:  
            df['Piston Velocity [m/s]'][i - 1] = (df['Piston Velocity [mm/deg]'][i - 1] *(RPM * 6)) / 1000
        
        if i >= 2:
    # Piston Acc [mm/deg/deg]:      
            df['Piston Acc [mm/deg/deg]'][i - 2] = df['Piston Velocity [mm/deg]'][i - 1] - df['Piston Velocity [mm/deg]'][i - 2]
            
    # Piston Acc [m/s/s]:
            df['Piston Acc [m/s/s]'][i - 2] = (df['Piston Acc [mm/deg/deg]'][i -2] / 1000) * (RPM * 6)**2
        
    ############ ########## ########## ########## ############ #########
    #Volume
    for i in range(len(df['Crank Angle'])): 

    # Volume [CC]:
        
        df['Volume [CC]'][i] = (Vcv +((math.pi * (Bore ** 2)) / 4) *
                                (ConRodLength + (0.5 * Stroke) - ((0.5 * Stroke) * 
                                                                 math.cos(math.radians(df['Crank Angle'][i])) +
                                                                 ((ConRodLength **2) -
                                                                  ((0.5 * Stroke)**2) *
                                                                  (math.sin(math.radians(df['Crank Angle'][i])) *
                                                                   (math.sin(math.radians(df['Crank Angle'][i])))))**0.5)))*1000000 
        
    ############ ########## ########## ########## ############ #########
    #CompressionPressure
    for i in range(len(df['Crank Angle'])):
        
    # Pressure [Bar]:
        if i < 1:
            df['Pressure [Bar]'][i] = atmospheric_pressure 
        elif i >= 1:
            df['Pressure [Bar]'][i] = df['Pressure [Bar]'][i - 1] * (df['Volume [CC]'][i - 1] / df['Volume [CC]'][i]) ** n_compression
            
    ############ ########## ########## ########## ############ #########   
    #MassFractionBurn
    for i in range(len(df['Crank Angle'])):
                    
    # Wiebe:
        df['Wiebe'][i] = 1 - np.exp(-Wiebe_a *((df['Crank Angle'][i] - (Ignition_deg + Ignition_Del))/ Bdur)**(Wiebe_m + 1))
        if df['Wiebe'][i] <= 0:
            df['Wiebe'][i] = 0
            
    # MFB:
        if i >= 1:
            df['MFB'][i -1] = df['Wiebe'][i] - df['Wiebe'][i - 1]
            
    ############ ########## ########## ########## ############ ######### 
    #HeatQ
    for i in range(len(df['Crank Angle'])):
    # dQr [J]:
                         
        df['dQr [J]'][i] = df['MFB'][i] * mtf * CombEfficiency * Calorific_fuel
        
    ############ ########## ########## ########## ############ ######### 
    #ExpansionPressure
    for i in range(len(df['Crank Angle'])):
    
    # Combustion [Bar]:
        if i == 0:
            df['Combustion [bar]'][i] = atmospheric_pressure 
        else:
            df['Combustion [bar]'][i] = (df['Combustion [bar]'][i - 1] * \
            (df['Volume [CC]'][i - 1] / df['Volume [CC]'][i])** n_compression)+(1/100000)* \
            (df['dQr [J]'][i]*(Gamma-1)/(df['Volume [CC]'][i]/1000000))
            
    ############ ########## ########## ########## ############ ######### 
    #CylinderTemp
    for i in range(len(df['Crank Angle'])):
        
    # Temperature [K]:
        df['Temperature [K]'][i] = ((df['Combustion [bar]'][i] *100000)*(df['Volume [CC]'][i]/1000000))/((mta+mtf)*R)
    
    ############ ########## ########## ########## ############ ######### 
    #workDone
    for i in range(len(df['Crank Angle'])):
    
    # Work Done [J]:
        if i == 0:
            df['Work Done [J]'][i] = 0
        elif i >= 1:
            df['Work Done [J]'][i] = (df['Combustion [bar]'][i] *100000)*((df['Volume [CC]'][i]-df['Volume [CC]'][i - 1])/1000000)
    
    ############ ########## ########## ########## ############ #########     
    # Table to get the engine results
    TotalWorkDone = sum(df['Work Done [J]'])
    PowerKw = (0.5 * n_cylinders) * (TotalWorkDone *((rpm / 60)))/1000  
    BHP = PowerKw / 0.74
    EngineCapacity = (((math.pi * (0.5 * (bore / 1000))**2) * (stroke / 1000))) *1000000 * n_cylinders
    Qr = sum(df['dQr [J]'])
    
    # loop through the keys in dataResults['Engine Cycle Simulation'] and update the corresponding value in dataResults['Results']
    for i, key in enumerate(dataResults['Engine Cycle Simulation']):
        if key == 'Work Done (Joules)':
            dataResults['Results'][i] = round(TotalWorkDone,1)
        elif key == 'Power (Kw)':
            dataResults['Results'][i] = round(PowerKw,1)
        elif key == 'BHP':
            dataResults['Results'][i] = round(BHP,1)
        elif key == 'Capacity (cc)':
            dataResults['Results'][i] = round(EngineCapacity,1)
        elif key == 'Qr (Joules)':
            dataResults['Results'][i] = round(Qr,1)
    
    return (df, dataResults)
            
########################################################################################################    
# Update concentration if is not 100%.
def checkWhatFuelis(FuelOption1, Concentration):
    x_interpol = 0
    
    if FuelOption1 == 24: #nButanol
        x_100 = 3  
        Concentration = 100
        a_interpol = np.interp(Concentration, [1, 100], [x_100 / 100, x_100]) # obtatin the maximum x for any concentration
    
    
    elif FuelOption1 == 23: #isoButanol
        x_100 = 6  
        Concentration = 100
        a_interpol = np.interp(Concentration, [1, 100], [x_100 / 100, x_100]) # obtatin the maximum x for any concentration
    
    elif FuelOption1 == 21: #hydrogen
        x_100 = 0.5  
        Concentration = 100
        a_interpol = np.interp(Concentration, [1, 100], [x_100 / 100, x_100]) # obtatin the maximum x for any concentration

    elif FuelOption1 == 20: #Ethanol **
        x_100 = 3  
        Concentration = 100
        a_interpol = np.interp(Concentration, [1, 100], [x_100 / 100, x_100])
        
    elif FuelOption1 == 19: # methanol
        x_100 = 1.5 
        Concentration = 100
        a_interpol = np.interp(Concentration, [1, 100], [x_100 / 100, x_100])
    
    elif FuelOption1 == 18: #isoDodecane
        x_100 = 18.5  
        Concentration = 100
        a_interpol = np.interp(Concentration, [1, 100], [x_100 / 100, x_100])
        
    elif FuelOption1 == 17: #isoDecane
        x_100 = 15.5  
        Concentration = 100
        a_interpol = np.interp(Concentration, [1, 100], [x_100 / 100, x_100])
    
    elif FuelOption1 == 16: #nDecane
        x_100 = 15.5  
        Concentration = 100
        a_interpol = np.interp(Concentration, [1, 100], [x_100 / 100, x_100])
        
    elif FuelOption1 == 15: #nNonane **
        x_100 = 14.0
        Concentration = 100
        a_interpol = np.interp(Concentration, [1, 100], [x_100 / 100, x_100])  

    elif FuelOption1 == 14: #isoOctane
        x_100 = 12.5
        Concentration = 100
        a_interpol = np.interp(Concentration, [1, 100], [x_100 / 100, x_100])
        
    elif FuelOption1 == 13: #nOctane
        x_100 = 12.5  
        Concentration = 100
        a_interpol = np.interp(Concentration, [1, 100], [x_100 / 100, x_100])
    
    elif FuelOption1 == 12: #nHeptane **
        x_100 = 11  
        Concentration = 100
        a_interpol = np.interp(Concentration, [1, 100], [x_100 / 100, x_100])
        
    elif FuelOption1 == 11: #isoHexane
        x_100 = 9.5  
        Concentration = 100
        a_interpol = np.interp(Concentration, [1, 100], [x_100 / 100, x_100])
    
    elif FuelOption1 == 10: #nHexane
        x_100 = 9.5  
        Concentration = 100
        a_interpol = np.interp(Concentration, [1, 100], [x_100 / 100, x_100])
        
    elif FuelOption1 == 9: #isoPentane **
        x_100 = 8  
        Concentration = 100
        a_interpol = np.interp(Concentration, [1, 100], [x_100 / 100, x_100])
    
    elif FuelOption1 == 8: #nPentane **
        x_100 = 8  
        Concentration = 100
        a_interpol = np.interp(Concentration, [1, 100], [x_100 / 100, x_100])
        
    elif FuelOption1 == 7: #isoButane
        x_100 = 6.5
        Concentration = 100
        a_interpol = np.interp(Concentration, [1, 100], [x_100 / 100, x_100])
    
    elif FuelOption1 == 6: #nButane
        x_100 = 6.5  
        Concentration = 100
        a_interpol = np.interp(Concentration, [1, 100], [x_100 / 100, x_100])
        
    elif FuelOption1 == 5: #propane ** 
        x_100 = 5 
        Concentration = 100
        a_interpol = np.interp(Concentration, [1, 100], [x_100 / 100, x_100])
    
    elif FuelOption1 == 4: #ethane
        x_100 = 3.5  
        Concentration = 100
        a_interpol = np.interp(Concentration, [1, 100], [x_100 / 100, x_100])
        
    elif FuelOption1 == 3: #ethylene **
        x_100 = 3  
        Concentration = 100
        a_interpol = np.interp(Concentration, [1, 100], [x_100 / 100, x_100])
    
    elif FuelOption1 == 2: #acetylene
        x_100 = 2.25  
        Concentration = 100
        a_interpol = np.interp(Concentration, [1, 100], [x_100 / 100, x_100])
        
    elif FuelOption1 == 1: #methane **
        x_100 = 2 
        Concentration = 100
        a_interpol = np.interp(Concentration, [1, 100], [(x_100 / 100), x_100])
    else:
        a_interpol = None
        
    return  a_interpol
########################################################################################################    
# Get the rate of change for pressure and combustion to gather where nox is likely to be created.
def RateChangePressure(df):
    for i in range(len(df['Crank Angle'])):
        if i == 0:
            df['Rate of Change Pressure'][i] = 0
            df['Rate of Change Combustion'][i] = 0
            df['Rate of Change Pressure - Combustion'][i] = 0
        else:
            df['Rate of Change Pressure'][i] = df['Pressure [Bar]'][i] - df['Pressure [Bar]'][i-1]
            df['Rate of Change Combustion'][i] = df['Combustion [bar]'][i] - df['Combustion [bar]'][i-1]
            df['Rate of Change Pressure - Combustion'][i] =  df['Rate of Change Combustion'][i] -  df['Rate of Change Pressure'][i]

    new_df = df.loc[abs(df['Rate of Change Pressure - Combustion'] )> 1, ['Crank Angle', 'Volume [CC]', 'Combustion [bar]']]
    new_df = new_df.reset_index(drop=True)
    new_df['Temp [K]'] = 0.0
    new_df['NoX [kmols/°]'] = 0.0
    return new_df

########################################################################################################    
# Obtain the Carbon considering thw whole equation
def get_Carbon_BalanceEq(eq1, eq2, eq3, Concentration1Eq, Concentration2Eq, Concentration3Eq):
# carbon n1
        # search for pattern C_{n} and C in eq1

        match_eq1 = re.findall(r'C_{(\d+)}|C([^_{]+|$)', eq1)
        # Check if any matches were found
        if match_eq1:
            count_c_eq1 = sum([int(i[0]) if i[0] != '' else len(i[1]) for i in match_eq1])
        else:
            # If no matches were found, count the number of H atoms in the equation
            count_c_eq1 = len(re.findall(r'C', eq1))

        # search for pattern C_{n} and C in eq2
        match_eq2 = re.findall(r'C_{(\d+)}|C([^_{]+|$)', eq2)
        # Check if any matches were found
        if match_eq2:
            count_c_eq2 = sum([int(i[0]) if i[0] != '' else len(i[1]) for i in match_eq2])
        else:
            # If no matches were found, count the number of H atoms in the equation
            count_c_eq2 = len(re.findall(r'C', eq2))

        # search for pattern C_{n} and C in eq3
        match_eq3 = re.findall(r'C_{(\d+)}|C([^_{]+|$)', eq3)
        # Check if any matches were found
        if match_eq3:
            count_c_eq3 = sum([int(i[0]) if i[0] != '' else len(i[1]) for i in match_eq3])
        else:
            # If no matches were found, count the number of H atoms in the equation
            count_c_eq3 = len(re.findall(r'C', eq3))

        totalCarbon = 0

        if count_c_eq1 > 0:
            totalCarbon += count_c_eq1 * Concentration1Eq

        if count_c_eq2 > 0:
            totalCarbon += count_c_eq2 * Concentration2Eq

        if count_c_eq3 > 0:
            totalCarbon += count_c_eq3 * Concentration3Eq
        return totalCarbon
    
# Obtain the Hydrogen considering thw whole equation    
def get_Hydrogen_BalanceEq(eq1, eq2, eq3, Concentration1Eq, Concentration2Eq, Concentration3Eq):
    # hydrogen n2
        # search for pattern H_{n} in eq1
        match_eq1 = re.findall(r'H_{(\d+)}|H([^_{]+|$)', eq1)

        # Check if any matches were found
        if match_eq1:
            count_h_eq1 = sum([int(i[0]) if i[0] != '' else len(i[1]) for i in match_eq1])
        else:
            # If no matches were found, count the number of H atoms in the equation
            count_h_eq1 = len(re.findall(r'H', eq1))

        # search for pattern H_{n} in eq2
        match_eq2 = re.findall(r'H_{(\d+)}|H([^_{]+|$)', eq2)
        # Check if any matches were found
        if match_eq2:
            count_h_eq2 = sum([int(i[0]) if i[0] != '' else len(i[1]) for i in match_eq2])
        else:
            # If no matches were found, count the number of H atoms in the equation
            count_h_eq2 = len(re.findall(r'H', eq2))

        # search for pattern H_{n} in eq3
        match_eq3 = re.findall(r'H_{(\d+)}|H([^_{]+|$)', eq3)

        # Check if any matches were found
        if match_eq3:
            count_h_eq3 = sum([int(i[0]) if i[0] != '' else len(i[1]) for i in match_eq3])
        else:
            # If no matches were found, count the number of H atoms in the equation
            count_h_eq3 = len(re.findall(r'H', eq3))

        totalHydrogen = 0

        if count_h_eq1 > 0:
            totalHydrogen += count_h_eq1 * Concentration1Eq

        if count_h_eq2 > 0:
            totalHydrogen += count_h_eq2 * Concentration2Eq

        if count_h_eq3 > 0:
            totalHydrogen += count_h_eq3 * Concentration3Eq

        totalHydrogen = totalHydrogen / 2
        return totalHydrogen
    
# Obtain the Oxygen considering thw whole equation
def get_Oxygen_BalanceEq(eq1, eq2, eq3, Concentration1Eq, Concentration2Eq, Concentration3Eq):
# Oxygen x
        # search for pattern C_{n} and O in eq1
        match_eq1 = re.findall(r'O_{(\d+)}', eq1)
        # Check if any matches were found
        if match_eq1:
            count_o_eq1 = sum([int(i[0]) if i[0] != '' else len(i[1]) for i in match_eq1])

        else:
            # If no matches were found, count the number of H atoms in the equation
            count_o_eq1 = len(re.findall(r'O', eq1))

        # search for pattern O_{n} and O in eq2
        match_eq2 = re.findall(r'O_{(\d+)}', eq2)
        # Check if any matches were found
        if match_eq2:
            count_o_eq2 = sum([int(i[0]) if i[0] != '' else len(i[1]) for i in match_eq2])

        else:
            # If no matches were found, count the number of H atoms in the equation
            count_o_eq2 = len(re.findall(r'O', eq2))

        # search for pattern O_{n} and O in eq3
        match_eq3 = re.findall(r'O_{(\d+)}', eq3)
        # Check if any matches were found
        if match_eq3:
            count_o_eq3 = sum([int(i[0]) if i[0] != '' else len(i[1]) for i in match_eq3])
        else:
            # If no matches were found, count the number of H atoms in the equation
            count_o_eq3 = len(re.findall(r'O', eq3))

        totalOxygen = 0

        if count_o_eq1 > 0:
            totalOxygen += count_o_eq1 * Concentration1Eq

        if count_o_eq2 > 0:
            totalOxygen += count_o_eq2 * Concentration2Eq

        if count_o_eq3 > 0:
            totalOxygen += count_o_eq3 * Concentration3Eq
            
        return totalOxygen
    
# Obtain the Nitrogen considering thw whole equation
def get_Nitrogen_BalanceEq(eq1, eq2, eq3, Concentration1Eq, Concentration2Eq, Concentration3Eq, totalOxygen): 
# Nitrogen n3
        # search for pattern C_{n} and N in eq1
        match_eq1 = re.findall(r'N_{(\d+)}|N([^_{]+|$)', eq1)
        # Check if any matches were found
        if match_eq1:
            count_n_eq1 = sum([int(i[0]) if i[0] != '' else len(i[1]) for i in match_eq1])
        else:
            # If no matches were found, count the number of H atoms in the equation
            count_n_eq1 = len(re.findall(r'N', eq1))

        # search for pattern N_{n} and N in eq2
        match_eq2 = re.findall(r'N_{(\d+)}|N([^_{]+|$)', eq2)
        # Check if any matches were found
        if match_eq2:
            count_n_eq2 = sum([int(i[0]) if i[0] != '' else len(i[1]) for i in match_eq2])
        else:
            # If no matches were found, count the number of H atoms in the equation
            count_n_eq2 = len(re.findall(r'N', eq2))

        # search for pattern N_{n} in eq3
        match_eq3 = re.findall(r'N_{(\d+)}|N([^_{]+|$)', eq3)
        # Check if any matches were found
        if match_eq3:
            count_n_eq3 = sum([int(i[0]) if i[0] != '' else len(i[1]) for i in match_eq3])
        else:
            # If no matches were found, count the number of H atoms in the equation
            count_n_eq3 = len(re.findall(r'N', eq3))

        totalNitrogen = 0

        if count_n_eq1 > 0:
            totaltotalNitrogen += count_n_eq1 * Concentration1Eq

        if count_n_eq2 > 0:
            totalNitrogen += count_n_eq2 * Concentration2Eq

        if count_n_eq3 > 0:
            totalNitrogen += count_n_eq3 * Concentration3Eq

        totalNitrogen = 3.76 * totalOxygen 
        return totalNitrogen

# check if the coefficients on the right side of the equation are integer, float or None
def optimize_value(value):
    if value is not None and value == round(value):
        return int(value)
    elif value is not None and 0 < value < 1:
        return round(value, 1)
    elif value == 1:
        return None
    else:
        return round(float(value), 2)
########################################################################################################    

def get_Carbon_FullBalance(eq1):
        # Remove everything after $$ if present
        if "$$" in eq1:
            eq1 = eq1.split("$$")[0]
    # carbon n1
        # search for pattern C_{n} and C in eq1

        match_eq1 = re.findall(r'C_{(\d+)}|C([^_{]+|$)', eq1)
        # Check if any matches were found
        if match_eq1:
            count_c_eq1 = sum([int(i[0]) if i[0] != '' else len(i[1]) for i in match_eq1])
        else:
            # If no matches were found, count the number of H atoms in the equation
            count_c_eq1 = len(re.findall(r'C', eq1))

        totalCarbon = 0

        if count_c_eq1 > 0:
            totalCarbon += count_c_eq1

        return totalCarbon

def get_Hydrogen_FullBalance(eq1):
    # Find all occurrences of H_{number}, H pattern not followed by _ or {
    # , }OH$ pattern, and }OH$$ pattern
    match_eq1 = re.findall(r'H_{(\d+)}|H([^_{]|$)|}OH\$|}OH\$\$', eq1)
    
    # If no matches were found, count the number of H atoms in the equation
    if not match_eq1:
        count_h_eq1 = len(re.findall(r'H', eq1))
    else:
        count_h_eq1 = sum([int(i) if i.isdigit() else 1 for group in match_eq1 for i in group if i]) 
    
    return count_h_eq1
    
def get_Oxygen_FullBalance(eq1):
            # Remove everything after $$ if present
        if "$$" in eq1:
            eq1 = eq1.split("$$")[0]
# Oxygen x
        # search for pattern C_{n} and O in eq1
        match_eq1 = re.findall(r'O_{(\d+)}', eq1)
        # Check if any matches were found
        if match_eq1:
            count_o_eq1 = sum([int(i[0]) if i[0] != '' else len(i[1]) for i in match_eq1])

        else:
            # If no matches were found, count the number of H atoms in the equation
            count_o_eq1 = len(re.findall(r'O', eq1))

        totalOxygen = 0

        if count_o_eq1 > 0:
            totalOxygen += count_o_eq1 
            
        return totalOxygen
    
def get_Nitrogen_FullBalance(eq1):
        # Remove everything after $$ if present
        if "$$" in eq1:
            eq1 = eq1.split("$$")[0]
# Nitrogen n3
        # search for pattern C_{n} and N in eq1
        match_eq1 = re.findall(r'N_{(\d+)}|N([^_{]+|$)', eq1)
        # Check if any matches were found
        if match_eq1:
            count_n_eq1 = sum([int(i[0]) if i[0] != '' else len(i[1]) for i in match_eq1])
        else:
            # If no matches were found, count the number of H atoms in the equation
            count_n_eq1 = len(re.findall(r'N', eq1))

        totalNitrogen = 0

        if count_n_eq1 > 0:
            totaltotalNitrogen += count_n_eq1
        return totalNitrogen
    
def get_airMultiplier(eq1):
    match = re.search(r'\+ (\d+(\.\d+)?)\(', eq1)
    if match:
        airMultiplier = float(match.group(1))
    return airMultiplier
########################################################################################################    

def getCoef_CO2(eq):
#     Find the coefficient in front of CO_2
    coefficient = re.findall(r'\b\d*(?:\.\d+)?(?=\s*CO_2\b)', eq)

    # If CO_2 has no coefficient, set it to 1
    if not coefficient:
        # Check if there's no CO2 in the equation, in which case set coefficient to 0
        if "CO_2" not in eq:
            coefficient_CO2 = 0
        else:
            coefficient_CO2 = 1
    else:
        # Convert the coefficient to a float
        coefficient_CO2 = float(coefficient[0]) if coefficient[0] != '' else 1
    return coefficient_CO2
    
# # # Enthalpy right side
def multiply_coefficients(dictionary, equation):
    total = 0
    for reactant in equation.split("+"):
        reactant = reactant.strip()
        match = re.match(r'([\d\.]+)?\s*([A-Za-z0-9_]+)', reactant)
        if match:
            coefficient = float(match.group(1) or 1)
            substance = match.group(2)
            if substance in dictionary:
                total -= coefficient * dictionary[substance]
    
    return total


def calculate_enthalpy_products(equation, inverted_dict):
    equation_0 = equation.split("$+")
    # Split the equation into its components
    components = equation_0[0].split(' + ')

    # Calculate the enthalpy change of the reaction
    enthalpy_change = 0
    for component in components:
        # Find the coefficient and the formula
        if ' ' not in component:
            coefficient = '1'
            formula = component
        else:
            coefficient, formula = component.split(' ', 1)

        # Check if the formula is in the inverted_dict
        if formula in inverted_dict:
            # Calculate the enthalpy contribution and add it to the total
            enthalpy_contribution = float(coefficient) * inverted_dict[formula]
            enthalpy_change += enthalpy_contribution
    return enthalpy_change

########################################################################################################    
# TEMP By guess


tempK_co2 = np.array([298, 300, 310, 320, 330, 340, 350, 360, 370, 380, 390, 400, 410, 420, 430, 440, 450, 460, 
                      470, 480, 490, 500, 510, 520, 530, 540, 550, 560, 570, 580, 590, 600, 610, 620, 630, 640,
                      650, 660, 670, 680, 690, 700, 710, 720, 730, 740, 750, 760, 770, 780, 790, 800, 810, 820, 
                      830, 840, 850, 860, 870, 880, 890, 900, 910, 920, 930, 940, 950, 960, 970, 980, 990, 1000, 
                      1020, 1040, 1060, 1080, 1100, 1120, 1140, 1160, 1180, 1200, 1220, 1240, 1260, 1280, 1300, 
                      1320, 1340, 1360, 1380, 1400, 1420, 1440, 1460, 1500, 1520, 1540, 1560, 1580, 1600, 1620, 
                      1640, 1660, 1680, 1700, 1720, 1740, 1760, 1780, 1800, 1820, 1840, 1860, 1880, 1900, 1920, 
                      1940, 1960, 1980, 2000, 2050, 2100, 2150, 2200, 2250, 2300, 2350, 2400, 2450, 2500, 2550, 
                      2600, 2650, 2700, 2750, 2800, 2850, 2900, 3000])

enthalpyco2 = np.array([0.000, 0.067, 0.443, 0.822, 1.206, 1.595, 1.987, 2.384, 2.784, 3.188, 3.596,
    4.008, 4.423, 4.842, 4.964, 5.690, 6.119, 6.552, 6.987, 7.427, 7.868, 8.314,
    8.762, 9.212, 9.665, 10.121, 10.581, 11.043, 11.506, 11.973, 12.443, 12.916,
    13.390, 13.867, 14.345, 14.826, 15.310, 15.796, 16.284, 16.774, 17.267, 17.761,
    18.258, 18.757, 19.258, 19.760, 20.265, 20.771, 21.280, 21.790, 21.801, 22.815,
    23.330, 23.848, 24.366, 24.887, 25.409, 25.932, 26.457, 26.983, 27.512, 28.041,
    28.571, 29.103, 29.636, 30.171, 30.706, 31.243, 31.781, 32.321, 32.862, 33.432,
    34.495, 35.589, 36.687, 37.789, 38.894, 40.005, 41.120, 42.238, 43.060, 44.484,
    45.613, 46.744, 47.880, 49.017, 50.158, 51.302, 52.449, 53.599, 54.752, 55.907,
    57.063, 58.222, 59.384, 61.714, 62.882, 64.053, 65.226, 66.403, 67.580, 68.759,
    69.939, 71.122, 72.306, 73.492, 74.679, 75.867, 77.056, 78.248, 79.442, 80.636,
    81.832, 83.030, 84.229, 85.429, 86.631, 87.833, 89.037, 90.242, 91.440, 94.471,
    97.500, 100.534, 103.575, 106.620, 109.671, 112.727, 115.788, 118.855, 121.926,
    125.004, 128.085, 131.169, 134.256, 137.349, 140.444, 143.544, 146.645, 152.862])


tempK_o2 = np.array([298, 300, 310, 320, 330, 340, 350, 360, 370, 380, 390, 400, 410, 420, 430, 440, 
          450, 460, 470, 480, 490, 500, 510, 520, 530, 540, 550, 560, 570, 580, 590, 600, 
          610, 620, 630, 640, 650, 660, 670, 680, 690, 700, 710, 720, 730, 740, 750, 760, 
          770, 780, 790, 800, 810, 820, 830, 840, 850, 860, 870, 880, 890, 900, 910, 920, 
          930, 940, 950, 960, 970, 980, 990, 1000, 1040, 1060, 1080, 1100, 1120, 1140, 1160, 
          1180, 1200, 1220, 1240, 1260, 1280, 1300, 1320, 1340, 1360, 1380, 1400, 1420, 1440, 
          1460, 1480, 1500, 1520, 1540, 1560, 1580, 1600, 1620, 1640, 1660, 1680, 1700, 1720, 
          1740, 1800, 1820, 1840, 1860, 1880, 1900, 1920, 1940, 1960, 1980, 2000, 2050, 2100, 
          2150, 2200, 2250, 2300, 2350, 2400, 2450, 2500, 2550, 2600, 2650, 2700, 2750, 2800, 
          2850, 2900, 3000])


enthalpy_o2 = np.array([0.000, 0.054, 0.348, 0.643, 0.938, 1.234, 1.531, 1.829, 2.127, 2.427,
                    2.727, 3.029, 3.330, 3.632, 3.936, 4.241, 4.546, 4.843, 5.160, 5.469,
                    5.778, 6.088, 6.400, 6.713, 7.026, 7.340, 7.656, 7.972, 8.289, 8.608,
                    8.927, 9.247, 9.568, 9.890, 10.213, 10.537, 10.862, 11.188, 11.515,
                    11.842, 12.172, 12.502, 12.832, 13.163, 13.495, 13.828, 14.162, 14.496,
                    14.831, 15.168, 15.504, 15.841, 16.179, 16.517, 16.855, 17.195, 17.536,
                    17.877, 18.217, 18.560, 18.902, 19.246, 19.590, 19.934, 20.278, 20.624,
                    20.970, 21.317, 21.663, 22.010, 22.359, 22.720, 24.107, 24.808, 25.512,
                    26.217, 26.924, 27.632, 28.341, 29.052, 29.765, 30.480, 31.195, 31.912,
                    32.630, 33.351, 34.071, 34.793, 35.516, 36.241, 36.966, 37.692, 38.420,
                    39.149, 39.879, 40.610, 41.342, 42.074, 42.808, 43.542, 44.279, 45.014,
                    45.752, 46.490, 47.230, 47.970, 48.712, 49.454, 51.689, 52.436, 53.184,
                    53.934, 54.683, 55.434, 56.186, 56.938, 57.692, 58.445, 59.199, 61.090,
                    62.986, 64.891, 66.802, 68.715, 70.634, 72.561, 74.492, 76.430, 78.375,
                    80.322, 82.274, 84.234, 86.199, 88.170, 90.144, 92.126, 94.111, 98.098])

tempK_n2 = tempK_o2

enthalpy_n2 = np.array([
    0.0, 0.054, 0.345, 0.637, 0.928, 1.219, 1.511, 1.802, 2.094, 2.386, 
    2.678, 2.971, 3.263, 3.556, 3.849, 4.142, 4.436, 4.73, 5.024, 5.319,
    5.616, 5.912, 6.207, 6.503, 6.8, 7.097, 7.395, 7.694, 7.993, 8.293, 
    8.593, 8.9, 9.799, 10.103, 10.406, 10.711, 11.016, 11.322, 11.628, 
    11.935, 12.243, 12.551, 12.86, 13.17, 13.48, 13.791, 14.103, 14.416, 
    14.729, 15.045, 15.358, 15.673, 15.989, 16.305, 16.623, 16.941, 17.259,
    17.579, 17.899, 18.221, 18.541, 18.863, 19.185, 19.509, 19.832, 20.157, 
    20.482, 20.807, 21.134, 21.47, 22.115, 22.773, 23.432, 24.093, 24.757,
    25.423, 26.091, 26.761, 27.435, 28.108, 28.783, 29.46, 30.138, 30.819, 
    31.501, 32.184, 32.87, 33.558, 34.246, 34.936, 35.626, 36.319, 37.013, 
    37.708, 38.404, 39.102, 39.801, 40.499, 41.2, 41.902, 42.606, 43.311, 
    44.017, 44.724, 45.43, 46.138, 46.847, 48.269, 48.982, 49.694, 50.406,
    51.121, 51.835, 52.551, 53.267, 53.985, 54.712, 55.421, 56.141, 57.943,
    59.748, 61.557, 63.371, 65.187, 67.007, 68.827, 70.651, 72.48, 74.312,
    76.145, 77.981, 79.819, 81.659, 83.502, 85.345, 87.19, 89.036, 92.738
])

tempK_h2o = np.array([298, 300, 310, 320, 330, 340, 350, 360, 370, 380, 390,
                      400, 410, 420, 430, 440, 450, 460, 470, 480, 490, 500, 
                      510, 520, 530, 540, 550, 560, 570, 580, 590, 600, 610, 
                      620, 630, 640, 650, 660, 670, 680, 690, 700, 710, 720, 
                      730, 740, 750, 760, 770, 780, 790, 800, 810, 820, 830, 
                      840, 850, 860, 870, 880, 890, 900, 910, 920, 930, 940, 
                      950, 960, 970, 980, 990, 1000, 1020, 1040, 1060, 1080, 
                      1100, 1120, 1140, 1160, 1180, 1200, 1220, 1240, 1260, 
                      1280, 1300, 1320, 1340, 1360, 1380, 1400, 1420, 1440, 
                      1460, 1480, 1500, 1520, 1540, 1560, 1580, 1600, 1620, 
                      1640, 1660, 1680, 1700, 1720, 1740, 1760, 1780, 1800, 
                      1820, 1840, 1860, 1880, 1900, 1920, 1940, 1960, 1980, 
                      2000, 2050, 2100, 2150, 2200, 2250, 2300, 2350, 2400, 
                      2450, 2500, 2550, 2600, 2650, 2700, 2750, 2800, 2850, 
                      2900, 3000])


enthalpy_h2o = np.array([0, 0.062, 0.398, 0.735, 1.072, 1.41, 1.748, 2.088, 2.427, 2.768, 3.11, 3.452, 3.795, 4.139, 4.484, 4.83, 5.176, 5.524, 5.873, 6.222, 6.573, 6.924, 7.277, 7.63, 7.985, 8.341, 8.697, 9.055, 9.414, 9.774, 10.135, 10.498, 10.861, 11.226, 11.591, 11.958, 12.326, 12.696, 13.066, 13.438, 13.81, 14.184, 14.56, 14.936, 15.314, 15.693, 16.073, 16.454, 16.837, 17.221, 17.606, 17.992, 18.38, 18.768, 19.158, 19.55, 19.942, 20.336, 20.731, 21.128, 21.525, 21.924, 22.324, 22.725, 23.128, 23.532, 23.937, 24.343, 24.749, 25.157, 25.568, 25.978, 26.805, 27.638, 28.476, 29.319, 30.167, 31.019, 31.876, 32.738, 33.605, 34.476, 35.352, 36.233, 37.118, 38.008, 38.903, 39.803, 40.708, 41.617, 42.53, 43.447, 44.369, 45.294, 46.224, 47.158, 48.095, 49.038, 49.984, 50.934, 51.888, 52.844, 53.805, 54.771, 55.739, 56.71, 57.685, 58.663, 59.646, 60.631, 61.619, 62.609, 63.603, 64.602, 65.602, 66.607, 67.613, 68.623, 69.636, 70.651, 71.669, 72.689, 75.252, 77.849, 80.426, 83.036, 85.658, 88.295, 90.942, 93.604, 96.279, 98.964, 101.661, 104.379, 107.087, 109.813, 112.549, 115.294, 118.048, 120.813, 126.36])


tempK_co = tempK_h2o


enthalpy_co = np.array([0, 0.054, 0.345, 0.637, 0.928, 1.22, 1.512, 1.804, 2.096, 2.389, 2.682, 2.975, 3.269, 3.563, 
                     3.857, 4.152, 4.447, 4.743, 5.039, 5.336, 5.633, 5.931, 6.229, 6.528, 6.828, 7.128, 7.428, 
                     7.73, 8.032, 8.334, 8.638, 8.942, 9.246, 9.552, 9.858, 10.164, 10.472, 10.78, 11.089, 11.399, 
                     11.709, 12.021, 12.333, 12.646, 12.959, 13.274, 13.589, 13.904, 14.221, 14.539, 14.857, 15.175, 
                     15.495, 15.814, 16.134, 16.455, 16.777, 17.099, 17.422, 17.746, 18.071, 18.397, 18.723, 19.05, 
                     19.377, 19.706, 20.034, 20.364, 20.693, 21.024, 21.355, 21.686, 22.351, 23.019, 23.688, 24.36, 
                     25.033, 25.708, 26.385, 27.064, 27.737, 28.426, 29.111, 29.797, 30.485, 31.175, 31.865, 32.557, 
                     33.25, 33.944, 34.64, 35.358, 36.038, 36.739, 37.441, 38.144, 38.848, 39.553, 40.259, 40.966, 
                     41.675, 42.384, 43.094, 43.803, 44.515, 45.226, 45.94, 46.654, 47.37, 48.087, 48.804, 49.522, 
                     50.241, 50.96, 51.682, 52.403, 53.125, 53.847, 54.569, 55.292, 56.015, 56.739, 58.555, 60.375, 
                     62.195, 64.019, 65.847, 67.676, 69.509, 71.346, 73.183, 75.023, 76.868, 78.714, 80.561, 82.408, 
                     84.261, 86.115, 87.97, 89.826, 93.541])

def polyfit_CO2():
    #### CO_2
    poly_co2 = np.polyfit(tempK_co2, enthalpyco2, 4)
    trendpoly_co2 = np.poly1d(poly_co2)
    return trendpoly_co2
    
def polyfit_O2():
    ### O_2
    poly_o2 = np.polyfit(tempK_o2, enthalpy_o2, 4)
    trendpoly_o2 = np.poly1d(poly_o2)
    return trendpoly_o2
    
def polyfit_N2():
    #### N_2
    poly_n2 = np.polyfit(tempK_n2, enthalpy_n2, 4)
    trendpoly_n2 = np.poly1d(poly_n2)
    return trendpoly_n2
    
def polyfit_H2O():
    #### h2o
    poly_h2o = np.polyfit(tempK_h2o, enthalpy_h2o, 4)
    trendpoly_h2o = np.poly1d(poly_h2o)
    return trendpoly_h2o
    
def polyfit_CO():
    #### co
    poly_co = np.polyfit(tempK_co, enthalpy_co, 4)
    trendpoly_co = np.poly1d(poly_co)
    return trendpoly_co


def coeficiente_co2_func(var1):
    match_co2 = re.search(r'(\d*\.\d+|\d+)\s*CO_2|CO_2', var1)
    if match_co2:
        coeficiente_co2 = float(match_co2.group(1) or '1')
    else:
        coeficiente_co2 = 0
    return coeficiente_co2 

def coeficiente_co_func(var1):
    match_co = re.search(r'(\d*\.\d+|\d+)\s*CO|CO', var1)
    if match_co:
        coeficiente_co = float(match_co.group(1) or '1')
        if "CO_2" in var1:
            coeficiente_co = 0
    else:
        coeficiente_co = 0
    return coeficiente_co

def coeficiente_n2_func(var1):
    match_n2 = re.search(r'(\d*\.\d+|\d+)\s*N_2|N_2', var1)
    if match_n2:
        coeficiente_n2 = float(match_n2.group(1) or '1')
    else:
        coeficiente_n2 = 0
    return coeficiente_n2

def coeficiente_h2o_func(var1):
    match_h2o = re.search(r'(\d*\.\d+|\d+)\s*H_2O|H_2O', var1)
    if match_h2o:
        coeficiente_h2o = float(match_h2o.group(1) or '1')
    else:
        coeficiente_h2o = 0
    return coeficiente_h2o

def coeficiente_o2_func(var1):
    match_o2 = re.search(r'(?:(\d+(?:\.\d*)?)\s*)O_2', var1)
    if match_o2:
        coeficiente_o2 = float(match_o2.group(1) or '1')
    else:
        coeficiente_o2 = 0
    return coeficiente_o2
########################################################################################################    


def nOX(flame_Temp, volume_cylinder,Throttle_IN, InitialTemp, PHI_VALUE, balanceEq,
   comb_chamber_volume, RPM_Func, atmospheric_pressure, new_df):

    throttle = (Throttle_IN / 100) - .1
    RT_constant = 8314  #
    temp_ambiente = InitialTemp  # Kelvin
    pressure_room_barToPascal = atmospheric_pressure * pow(10,5) # atmospheric_pressure Pascal

    n = (pressure_room_barToPascal  *  volume_cylinder *  throttle ) / \
    (RT_constant * temp_ambiente ) #  the number of kmols of mixture draw
    ratioCurrent = PHI_VALUE

    equation_split = balanceEq.split(" \longrightarrow ")
    leftSideEq = equation_split[0]
    righttSideEq = equation_split[1]

    if '(O_2 + 3.76 N_2)' in leftSideEq:
        airMultiplier = get_airMultiplier(leftSideEq)

    xfrom_o2 = airMultiplier
    alpha = n / (ratioCurrent + xfrom_o2 * (1 + 3.76))

    o2 = coeficiente_o2_func(righttSideEq)
    n2 = coeficiente_n2_func(righttSideEq)
    co_2 = coeficiente_co2_func(righttSideEq)
    h_2o = coeficiente_h2o_func(righttSideEq)

    concentration_O2 = (alpha * o2) / (comb_chamber_volume)
    roundO2 = round(concentration_O2,6)
    concentration_N2 = (alpha * n2) / (comb_chamber_volume)
    roundN2 = round(concentration_N2,6)

    deltano_deltaT = (4.7 * pow(10,13) * roundN2 *
          pow(roundO2, .5) * 
          np.exp( -67837 / (flame_Temp) )) / (RPM_Func * 6)

    noX = round(deltano_deltaT * comb_chamber_volume,11 )
    Cv = 718                                                                                     # Specific heat @ constant volume for air J / kgK
    Cp = 1007
    polytropicIndex = Cp / Cv

    for i in range(len(new_df['Crank Angle'])):
        if i == 0:
            new_df['Temp [K]'][i] = flame_Temp
            new_df['NoX [kmols/°]'][i] = noX
        else:
            new_df['Temp [K]'][i] = new_df['Temp [K]'][i -1] * (new_df['Combustion [bar]'][i] / \
                        new_df['Combustion [bar]'][i - 1]) ** ((polytropicIndex - 1) / polytropicIndex)                    

            new_df['NoX [kmols/°]'][i] = ((4.7 * pow(10,13) * roundN2 *pow(roundO2, .5) * \
              np.exp( -67837 / (new_df['Temp [K]'][i]) )) / (RPM_Func * 6)) * comb_chamber_volume
            
    kmolsCycle = new_df['NoX [kmols/°]'].sum()
    totalExhaustKmols = alpha * (o2 + n2 + co_2 + h_2o)
    PPM = (totalExhaustKmols / kmolsCycle) * pow(10,-6)
    return new_df, kmolsCycle, totalExhaustKmols, PPM


In [6]:
########################### variables ###############################

fuel_1_variables=   [{'label': 'Methane', 'value': 1},
                     {'label': 'Acetylene', 'value': 2},
                     {'label': 'Ethylene', 'value': 3},
                     {'label': 'Ethane', 'value': 4},
                     {'label': 'Propane', 'value': 5},
                     {'label': 'n-Butane', 'value': 6},
                     {'label': 'iso-Butane', 'value': 7},
                     {'label': 'n-Pentane', 'value': 8},
                     {'label': 'iso-Pentane', 'value': 9},
                     {'label': 'n-Hexane', 'value': 10},
                     {'label': 'iso-Hexane', 'value': 11},
                     {'label': 'n-Heptane', 'value': 12},
                     {'label': 'n-Octane', 'value': 13},
                     {'label': 'iso-Octane', 'value': 14},
                     {'label': 'n-Nonane', 'value': 15},
                     {'label': 'n-Decane', 'value': 16},
                     {'label': 'iso-Decane', 'value': 17},
                     {'label': 'iso-Dodecane', 'value': 18},
                     {'label': 'Methanol', 'value': 19},
                     {'label': 'Ethanol', 'value': 20},
                     {'label': 'Hydrogen', 'value': 21},
                     {'label': 'None', 'value': 22},
                     {'label': 'isoButanol', 'value': 23},
                     {'label': 'nButanol', 'value': 24},
                    ]

############ # # # # ################## # # # ################## # # # ################## 

#################### Chemical formulas ####################

methaneEq = '$CH_{4}$'  
acetyleneEq = '$C_{2}H_2$' 
ethyleneEq = '$C_{2}H_{4}$'
ethaneEq = '$C_{2}H_{6}$'
propaneEq = '$C_{3}H_{8}$'
nButaneEq = '$C_{4}H_{10}$'
isoButaneEq = '$C_{4}H_{10}$'
nPentaneEq = '$C_{5}H_{12}$'
isoPentaneEq = '$C_{5}H_{12}$'
nHexaneEq = '$C_{6}H_{14}$'
isoHexaneEq = '$C_{6}H_{14}$'
nHeptaneEq = '$C_{7}H_{16}$'
nOctaneEq = '$C_{8}H_{18}$'
isoOctaneEq = '$C_{8}H_{18}$'
nNonaneEq = '$C_{9}H_{20}$'
nDecane = '$C_{10}H_{22}$'
isoDecaneEq = '$C_{12}H_{26}$' #Diesel
isoDodecaneEq = '$C_{12}H_{26}$'
methanolEq = '$CH_{3}OH$'
EthanolEq = '$C_{2}H_{5}OH$'
hydrogenEq = '$H_{2}$'
isoButanolEq = '$C_{4}H_{10}O$'
nButanolEq = '$C_4H_{9}OH$'

########################################################

# Fuel data (Tables from Combustion - Irvin Glassman, Richard A. Yetter, ISBN: 978-0-12-088573-2)
enthalpy_dataMJKmol = {
    'CO': -110.530, # (g)
    'CO_2': -393.520, # (g)
    'H_2O': -241.820, # (l)
    'H_2O_2': -136.310, #(g)
    'NH_3': -46.190, # (g)
    'CH_4': -74.850, # (g)
    'C_2H_2': 226.730, # (g)
    'C_2H_4': 52.280, # (g)
    'C_2H_6': -84.680, # (g)
    'C_3H_6': 20.410, # (g)
    'C_6H_6': 82.930, # (g)
    'O': 249.190, # (g)
    'H': 218, # (g)
    'N': 472.650, # (g)
    'OH': 39.460 # (g)
}


enthalpy_dataMJKmol_Liquid = {
    'CO': -110.530, # (g)
    'CO_2': -393.520, # (g)
    'H_2O': -285.820, # (l)
    'H_2O_2': -136.310, #(g)
    'NH_3': -46.190, # (g)
    'CH_4': -74.850, # (g)
    'C_2H_2': 226.730, # (g)
    'C_2H_4': 52.280, # (g)
    'C_2H_6': -84.680, # (g)
    'C_3H_6': 20.410, # (g)
    'C_6H_6': 82.930, # (g)
    'O': 249.190, # (g)
    'H': 218, # (g)
    'N': 472.650, # (g)
    'OH': 39.460 # (g)
}

inverted_dict = {
    '$CH_{4}$': -74.40,
    '$C_{2}H_2$': 227.060,
    '$C_{2}H_{4}$': 52.40,
    '$C_{2}H_{6}$': -83.8,
    '$C_{3}H_{8}$': -104.7,
    '$C_{4}H_{10}$': -146.6,
    '$C_{4}H_{10}$': -153.5,
    '$C_{5}H_{12}$': -173.5,
    '$C_{5}H_{12}$': -178.5,
    '$C_{6}H_{14}$': -198.7,
    '$C_{6}H_{14}$': -207.4,
    '$C_{7}H_{16}$': -224.2,
    '$C_{8}H_{18}$': -250.1,
    '$C_{8}H_{18}$': -259.2,
    '$C_{9}H_{20}$': -274.7,
    '$C_{10}H_{22}$': -300.9,
    '$C_{12}H_{26}$': -300.9,
    '$C_{12}H_{26}$': -350.9,
    '$CH_{3}OH$': -201.5,
    '$C_{2}H_{5}OH$': -235.1,
    '$H_{2}$': 0
}

In [7]:
######################################## DASHBOARD ########################################

# https://www.bootstrapcdn.com/bootswatch/
app = JupyterDash(__name__, external_stylesheets=[dbc.themes.COSMO],
                meta_tags=[{'name': 'viewport',
                            'content': 'width=device-width, initial-scale=1.0'}],
                  
                )

In [8]:
####################################  BUTTONS #############################


# Buttons for the engine Simulation https://dash-bootstrap-components.opensource.faculty.ai/docs/components/input/
# ************************************************************************
CylindersButton = html.Div([
        dbc.Label('Number Cylinders', size='lg'),
        dbc.Input(id='nCylindersVar', placeholder='I.e. 4', type='number', min=1),

            ])

ConRodLengthButton = html.Div([
        dbc.Label('Connecting Rod. mm', size='lg'),
        dbc.Input(id='conRodLengVar', placeholder='I.e. 100', type='number', min=1),

            ])

StrokeButton = html.Div([
        dbc.Label('Stroke in mm', size='lg'),
        dbc.Input(id='strokeVar', placeholder='I.e. 81.4', type='number', min=1),
            ])

BoreButton = html.Div([
        dbc.Label('Bore in mm', size='lg'),
        dbc.Input(id='boreVar', placeholder='I.e. 79', type='number', min=1),
            ])

CompressionButton = html.Div([
        dbc.Label('Compression ratio', size='lg'),
        dbc.Input(id='compVar', placeholder='I.e. 11.23', type='number', min=1),
            ])

TAmbientButton = html.Div([
        dbc.Label('Room temperature °C', size='lg'),
        dbc.Input(id='t_ambVar', placeholder='I.e. 25', type='number', min=1),
            ])

PAmbientButton = html.Div([
        dbc.Label('ATM Bar', size='lg'),
        dbc.Input(id='atmVar', placeholder='I.e. 1', type='number', min=1),
            ])

ThrottleButton = html.Div([
        dbc.Label('Throttle in %', size='lg'),
        dbc.Input(id='throttleVar', placeholder='I.e. 100', type='number', min=1, max=100),
            ])

RPMButton = html.Div([
        dbc.Label('RPM', size='lg'),
        dbc.Input(id='RPMVar', placeholder='I.e. 8000', type='number', min=1),
            ])

Ignition_DelayButton = html.Div([ 
        dbc.Label('Ignition Delay', size='lg'),
        dbc.Input(id='igDelayVar', placeholder='I.e. 8', type='number', min=1),
            ])

Ignition_degButton = html.Div([ 
        dbc.Label('Ignition Degree', size='lg'),
        dbc.Input(id='igdegVar', placeholder='I.e. -25', type='number', min=-80),
            ])

BdurButton = html.Div([ 
        dbc.Label('Burn duration in Deg', size='lg'),
        dbc.Input(id='burnDVar', placeholder='I.e. 35', type='number', min=1),
            ])

In [9]:
app.layout = dbc.Container([
    
html.Div([    
html.H1('Lean NOx Model', style={'color': '#0E86D4', 'text-align': 'left',}),
     
dbc.Row([
dbc.Col( 
html.Div([
              html.Label('Fuel Nº 1', style={'font-size': '15px', 
              'font-family': 'Arial', 'text-align': 'center'}),
    
              dcc.Dropdown(id='FuelOption-1',
              options= fuel_1_variables,   
              value= 22, 
              style={'width': '150%'}), #dcc.Dropdown
    
              dcc.Input(id='Concentration-1', type='number', placeholder='%',
              min=0, max=100, step=0.1, style={'text-align': 'center',
              'width': '80%',  'margin-top' : '4px', }), #dcc.Input
]), # div
),#dbc.Col
],style={'position': 'absolute', 'left': '25px', 'top': '80px'},),# dbc.Row

############ # # # # ################## # # # ################## # # # ##################
dbc.Row([
dbc.Col( 
html.Div([
              html.Label('Fuel Nº 2', style={'font-size': '15px', 
              'font-family': 'Arial', 'text-align': 'center'}),
    
              dcc.Dropdown(id='FuelOption-2',
              options= fuel_1_variables,   
              value= 22, 
              style={'width': '150%'}), #dcc.Dropdown
    
              dcc.Input(id='Concentration-2', type='number', placeholder='%',
              min=0, max=100, step=0.1, style={'text-align': 'center',
              'width': '80%',  'margin-top' : '4px', }), #dcc.Input
]), # div
),#dbc.Col
],style={'position': 'absolute', 'left': '300px', 'top': '80px'},),# dbc.Row
     
############ # # # # ################## # # # ################## # # # ##################
dbc.Row([
dbc.Col( 
html.Div([
              html.Label('Fuel Nº 3', style={'font-size': '15px', 
              'font-family': 'Arial', 'text-align': 'center'}),
    
              dcc.Dropdown(id='FuelOption-3',
              options= fuel_1_variables,   
              value= 22, 
              style={'width': '150%'}), #dcc.Dropdown
    
              dcc.Input(id='Concentration-3', type='number', placeholder='%',
              min=0, max=100, step=0.1, style={'text-align': 'center',
              'width': '80%',  'margin-top' : '4px', }), #dcc.Input
]), # div
),#dbc.Col
],style={'position': 'absolute', 'left': '580px', 'top': '80px'},),# dbc.Row
############ # # # # ################## # # # ################## # # # ##################

        # Reagents equation with no Air
dbc.Row([
dbc.Col(
html.Div(
        dcc.Markdown(id='balanceEq', mathjax=True)),#html.Div
        style={'width': '50%', 'position': 'absolute', 'left': '10px', 'top': '200px',
        'display': 'flex', 'flex-wrap': 'nowrap', 'flex-direction': 'row'}), #dbc.Col
 
dcc.Interval(
        id='interval-component',
        interval=1 * 1000,  # in milliseconds
        n_intervals = 0), #dcc.Interval
],),#dcc.Interval
    
############ # # # # ################## # # # ################## # # # ##################       
],), #html.Div

#Buttuns Left Side
    
dbc.Row([dbc.Col([
    CylindersButton,
    ConRodLengthButton,
    StrokeButton,
    BoreButton,
    CompressionButton,
    RPMButton
], width=16,),
        
         
],style={'position': 'absolute', 'top': '50px', 'left': '900px'},), 
#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-##-#-#-#-#-#-#-#
#Buttuns right Side
dbc.Row([dbc.Col([
    TAmbientButton,
    PAmbientButton,
    ThrottleButton,
    Ignition_DelayButton,
    Ignition_degButton,
    BdurButton,
], width=16, ),
],style={'position': 'absolute', 'top': '50px', 'left': '1150px'},),
    
# ############ ########## ########## ########## ############ #########
#Drop-Donw Menu 
#Reference-> https://github.com/27shraddhaS/dropdown-graphs-dash
dbc.Row([dbc.Col([
dcc.Dropdown(
            id='dropdown1',
            options=[{'label': 'Piston position (mm)', 'value': '1'},
            {'label': 'Piston Speed (m/s)', 'value': '2'},
            {'label': 'Piston Acceleration (m/s/s)', 'value': '3'},
            {'label': 'Cylinder Volume (CC)', 'value': '4'},
            {'label': 'Pressure (Bar)', 'value': '5'},
            {'label': 'Temperature (K)', 'value': '6'},
            {'label': 'Work Done [J]', 'value': '7'},
            {'label': 'MFB', 'value': '8'},
            {'label': 'Pressure (Bar) vs Volume (cc) Graph', 'value': '9'},
            {'label': 'Graphs', 'value': '10'},
                    ],
            value='10',),
 
    
],  ),
],style={'position': 'absolute', 'top': '740px', 'left': '20px', 'text-align': 'center','width': '800px'},),
# ############ ########## ########## ########## ############ #########   
# # Calculate engine 
dbc.Row([dbc.Col([
dbc.Button("Calcule", outline=True, color="warning", className="me-1", 
           size="lg",id='submit-buttonCalc', n_clicks = 0),
            html.Div(id='outputCalc') ],),#dbc.Col
            ],style={'position': 'absolute', 'top': '580px', 'left': '900px'},), 

# #-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-##-#-#-#-#-#-#-#
       
html.Div([
            html.Div(id='graph_engine',
            ),],style={'position': 'absolute', 'top': '785px', 'left': '10px', 'width': '50%'},),#html.Div
    
# #-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-##-#-#-#-#-#-#-#
     
#EGR
dcc.Store(id='slider_value_egr', data=0),
    dbc.Row([dbc.Col([   
 daq.Slider(
            id='slider',
            min=0,
            max=100,
            value=0,
            handleLabel={'showCurrentValue': True,'label': 'EGR %'},
            step=1,
            vertical=True,
            size=180,
),      
], style={'position': 'absolute', 'top': '300px', 'left': '800px', 'padding-right': '0px'}  ),
            ],),  
    
            html.Script('''
            setInterval(() => {
            var slider = document.getElementById('slider');
            slider.value = Math.floor(Math.random() * 100);
            }, 1000);
            '''),
 ############ # # # # ################## # # # ################## # # # ##################
    # phi
 dcc.Store(id='slider_value_phi', data=0),
 dbc.Row([dbc.Col([
 daq.Slider(
            id = 'slider_phi',
            min=0.3,
            max=1.9,
            value=1,
            handleLabel={'showCurrentValue': True,'label': 'Φ Air Fuel Ratio'},
            step=.01,
            vertical=True,
            size=180,
            marks={'0.1': 'Lean', '1': 'Stoichiometeric', '2.1': 'Rich',}
        ),      
        ], style={'position': 'absolute', 'top': '300px', 'left': '650px', 'padding-right': '0px'}  ),
                    ],),  
            html.Script('''
            setInterval(() => {
            var slider = document.getElementById('slider');
            slider.value = Math.floor(Math.random() * 100);
            }, 1000);
            '''),
# ############ # # # # ################## # # # ################## # # # ##################
dbc.Row([dbc.Col([
                 
dash_table.DataTable(
    id='table_results',
    columns=[{'name': i, 'id': i} for i in resultsnoX.columns],
    data = resultsnoX.to_dict('records'),
    style_as_list_view=True,
        style_table={
        'border': '0px',
        'width': '450px',
        'textOverflow': 'ellipsis',
        'overflow': 'hidden',
        },
        style_cell={
        'textAlign': 'left',
        'paddingRight': '0px',
    },
    style_header={
        'backgroundColor': 'white',
        'fontWeight': 'bold'
    },


),

],   ), ],style={'position': 'absolute', 'top': '240px', 'left': '15px', 
                 'background-color': 'transparent'},),  

    
    
# ############ # # # # ################## # # # ################## # # # ##################  

html.Div([        
    dbc.Alert(id='results_engine',color='light',
              children=generate_table(pd.DataFrame( columns=['Results', '  '], 
               data=[[k, v] for k, v in zip(dataResults['Engine Cycle Simulation'], 
                dataResults['Results'])]
                )
            ),
              style={'backgroundColor': 'rgba(255, 255, 255, 0.5)'}
        )
    ],
    style={
        'position': 'absolute',
        'top': '795px',
        'left': '710px',
    },
),

# ############ # # # # ################## # # # ################## # # # ################## 
    
html.Div([
            html.Div(id='graph_nOX',
            ),],style={'position': 'absolute', 'top': '765px', 
                       'left': '1100px', 'width': '60%'},),#html.Div
    
# ############ # # # # ################## # # # ################## # # # ################## 
    
# # Calculate engine 
dbc.Row([dbc.Col([
dbc.Button("Nox", outline=True, color='#FF4136', className="me-1", 
           size="lg",id='submit-buttonNox', n_clicks = 0),
            html.Div(id='outputCalc') ],),#dbc.Col
            ],style={'position': 'absolute', 'top': '735px', 'left': '810px'},), 
# #-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-##-#-#-#-#-#-#-#
], style={'height': '100vh'})
# ], )

In [10]:
# ############ # # # # ################## # # # ################## # # # ##################
#-3 EGR SLIDER
@app.callback(Output('slider_value_egr', 'data'),
              [Input('slider', 'value'),])
def store_slider_value_egr(value):
    if value is None:
        return 0
    else:
        value = 1 + (value / 100)
    return value
# ############ # # # # ################## # # # ################## # # # ##################  
#-4 phi SLIDER
@app.callback(Output('slider_value_phi', 'data'),
              [Input('slider_phi', 'value')])
def store_slider_value_phi(value):
    return value

# ############ # # # # ################## # # # ################## # # # ################## 
@app.callback(
    [Output('Concentration-1', 'value'),
    Output('Concentration-2', 'value'),
    Output('Concentration-3', 'value'),],
    
    [Input('interval-component', 'n_intervals'), 
     Input('FuelOption-1', 'value'), 
     Input('FuelOption-2', 'value'), 
     Input('FuelOption-3', 'value'), 
     Input('Concentration-1', 'value'),
     Input('Concentration-2', 'value'),
     Input('Concentration-3', 'value'),] 
)

def Concentration(n_intervals,FuelOption1, FuelOption2, FuelOption3, Concentration1, Concentration2, Concentration3):
    concentration = [Concentration1, Concentration2, Concentration3]
            
    if FuelOption1 != 22 and FuelOption2 == 22 and FuelOption3 == 22:
        if Concentration1 is not None:
            if Concentration1 > 100:
                Concentration1 = 100
        concentration = [Concentration1, None, None]  
        
    elif FuelOption1 != 22 and FuelOption2 != 22 and FuelOption3 == 22:
        if Concentration1 is not None :
            if Concentration1 >= 100:
                Concentration1 = 99 
                
            concentration = [Concentration1, 100 - Concentration1, None] 

    elif FuelOption1 != 22 and FuelOption2 != 22 and FuelOption3 != 22:
        if Concentration1 is not None and Concentration2 is not None  :
            
            if Concentration2 + Concentration1 >= 100:
                Concentration1 = 1
                Concentration2 = 1
            
            Concentration3 = 100 - (Concentration2 + Concentration1)
        
        concentration = [Concentration1, Concentration2, Concentration3]        
        
    return concentration

# ############ # # # # ################## # # # ################## # # # ##################    
#-6 Display Fuel Equations LateX
@app.callback(
    dash.dependencies.Output(component_id='balanceEq', component_property='children'),
    
    [Input(component_id='FuelOption-1', component_property='value'),
     Input(component_id='FuelOption-2', component_property='value'),
     Input(component_id='FuelOption-3', component_property='value'),
     Input('interval-component', 'n_intervals'), Input('Concentration-1', 'value'),
    Input('Concentration-2', 'value'), Input('Concentration-3', 'value'), 
    Input('slider_value_egr', 'data'), Input('slider_value_phi', 'data')]
)
def balanceEqLateX(FuelOption1,FuelOption2,FuelOption3, n_intervals, Concentration1,
                  Concentration2, Concentration3, EGR_VALUE, phiValue ):
    options = {
        1:  methaneEq, 
        2:  acetyleneEq, 
        3:  ethyleneEq, 
        4:  ethaneEq,
        5:  propaneEq,
        6:  nButaneEq,
        7:  isoButaneEq,
        8:  nPentaneEq,
        9:  isoPentaneEq,
        10: nHexaneEq,
        11: isoHexaneEq,
        12: nHeptaneEq,
        13: nOctaneEq,
        14: isoOctaneEq,
        15: nNonaneEq,
        16: nDecane,
        17: isoDecaneEq,
        18: isoDodecaneEq,
        19: methanolEq,
        20: EthanolEq,
        21: hydrogenEq,
        22: '',
        23: isoButanolEq,
        24: nButanolEq
    }
    result = ''
    for i, option in enumerate([FuelOption1, FuelOption2, FuelOption3]):
        if option in options:
            if i == 0 and option != 22:
                result += options[option]
            elif i == 1 and option != 22 and FuelOption1 != 22:
                result += ' + ' + options[option]
            elif i == 2 and option != 22:
                result += ' + ' + options[option]

    Concentration1Eq = 0
    Concentration2Eq = 0
    Concentration3Eq = 0
    FullConcentration = 100

    if Concentration1 is not None:
        Concentration1Eq = Concentration1 / 100
    if Concentration2 is not None:
        Concentration2Eq = Concentration2 / 100
    if Concentration3 is not None:
        Concentration3Eq = Concentration3 / 100
    eq1 = ''
    eq2 = ''
    eq3 = ''


#     equations = balanceEq.split(" + ")
    equations = result.split(" + ") 

    if len(equations) == 1:
        eq1 = equations[0]
        eq2 = ''
        eq3 = ''
    elif len(equations) == 2:
        eq1 = equations[0]
        eq2 = equations[1]
        eq3 = ''
    elif len(equations) == 3:
        eq1 = equations[0]
        eq2 = equations[1]
        eq3 = equations[2]

    if FuelOption1 != 22 and Concentration1 is not None:
        ############################################# stoichiometrically ####################################
        totalCarbon = get_Carbon_BalanceEq(eq1, eq2, eq3, Concentration1Eq, Concentration2Eq, Concentration3Eq)
        totalCarbonCheck = totalCarbon
        totalHydrogen = get_Hydrogen_BalanceEq(eq1, eq2, eq3, Concentration1Eq, Concentration2Eq, Concentration3Eq)
        totalOxygen = get_Oxygen_BalanceEq(eq1, eq2, eq3, Concentration1Eq, Concentration2Eq, Concentration3Eq)
        totalOxygenCheck = totalOxygen
        totalOxygen = ((2 * totalCarbon + totalHydrogen) - totalOxygen) / 2
        totalNitrogen = get_Nitrogen_BalanceEq(eq1, eq2, eq3, Concentration1Eq, Concentration2Eq, Concentration3Eq, totalOxygen)

        n1 = optimize_value(totalCarbon)
        n2 = optimize_value(totalHydrogen)
        x = optimize_value(totalOxygen)
        n3 = optimize_value(totalNitrogen)


    ################ STOICHOMETRICALLY ############
        if phiValue == 1: # stoichiometrically
#         if EGR_VALUE == 1:
            finalEq =  f'$+ {x}(O_{2} + 3.76 N_{2}) \longrightarrow  {n1} CO_{2} + {n2} H_{2}O + {n3} N_{2}$' #→

            if x == 0:
                finalEq = f'$+ \longrightarrow  {n1} CO_{2} + {n2} H_{2}O + {n3} N_{2}$'
            elif n1 == 0:
                finalEq = f'$+ {x}(O_{2} + 3.76 N_{2}) \longrightarrow {n2} H_{2}O + {n3} N_{2}$'
            elif n2 == 0 :
                finalEq = f'$+ {x}(O_{2} + 3.76 N_{2}) \longrightarrow  {n1} CO_{2} + {n3} N_{2}$'
            elif n3 == 0 :
                finalEq = f'$+ {x}(O_{2} + 3.76 N_{2}) \longrightarrow  {n1} CO_{2} + {n2} H_{2}O$'

            if EGR_VALUE > 1:
                finalEq =  f'$+ {x}(O_{2} + 3.76 N_{2}) \longrightarrow  {round(n1 * EGR_VALUE,2)} CO_{2} + {round(n2 * EGR_VALUE,2)} H_{2}O + {round(n3 * EGR_VALUE,2)} N_{2}$' #→
                if x == 0:
                    finalEq = f'$+ \longrightarrow  {round(n1 * EGR_VALUE,2)} CO_{2} + {round(n2 * EGR_VALUE,2)} H_{2}O + {round(n3 * EGR_VALUE,2)} N_{2}$'
                elif n1 == 0:
                    finalEq = f'$+ {x}(O_{2} + 3.76 N_{2}) \longrightarrow {round(n2 * EGR_VALUE,2)} H_{2}O + {round(n3 * EGR_VALUE,2)} N_{2}$'
                elif n2 == 0 :
                    finalEq = f'$+ {x}(O_{2} + 3.76 N_{2}) \longrightarrow  {round(n1 * EGR_VALUE,2)} CO_{2} + {round(n3 * EGR_VALUE,)} N_{2}$'
                elif n3 == 0 :
                    finalEq = f'$+ {x}(O_{2} + 3.76 N_{2}) \longrightarrow  {round(n1 * EGR_VALUE,2)} CO_{2} + {round(n2 * EGR_VALUE,2)} H_{2}O$'


# ################################## RICH ########################## #
        elif phiValue > 1:

            n3phiEqual1 = round((phiValue * totalHydrogen),2)
            OxygenTotalRich = (totalOxygen * 2) + totalOxygenCheck * phiValue
            n1Plusn2 = totalCarbonCheck * phiValue
            n1phiEqual1 = float(round((OxygenTotalRich - n3phiEqual1 - n1Plusn2),2))
            n2phiEqual1 = round((n1Plusn2 - n1phiEqual1),2)
            n4phiEqual1 = round((3.76 * totalOxygen),2)
            x = round(totalOxygen,2)

            
            if n1phiEqual1 <= 0 and FuelOption1 != 21 :
                finalEq =  f'$+ {x}(O_{2} + 3.76 N_{2}) \longrightarrow {n2phiEqual1} CO + {n3phiEqual1} H_{2}O + {n4phiEqual1} N_{2}$' #→
                
            elif FuelOption1 == 21 and FuelOption2 == 22 and FuelOption3 == 22:
                finalEq =  f'$+ {x}(O_{2} + 3.76 N_{2}) \longrightarrow {n3phiEqual1} H_{2}O + {n4phiEqual1} N_{2}$' #→
                
            elif FuelOption1 == 21 and FuelOption2 == 21:
                finalEq =  f'$+ {x}(O_{2} + 3.76 N_{2}) \longrightarrow {n3phiEqual1} H_{2}O + {n4phiEqual1} N_{2}$' #→
            elif FuelOption1 == 21 and FuelOption2 == 21 and FuelOption3 == 21:
                finalEq =  f'$+ {x}(O_{2} + 3.76 N_{2}) \longrightarrow {n3phiEqual1} H_{2}O + {n4phiEqual1} N_{2}$' #→
            else:
                if n1phiEqual1 <= 0:
                    finalEq =  f'$+ {x}(O_{2} + 3.76 N_{2}) \longrightarrow {n2phiEqual1} CO + {n3phiEqual1} H_{2}O + {n4phiEqual1} N_{2}$' #→
                else:
                    finalEq =  f'$+ {x}(O_{2} + 3.76 N_{2}) \longrightarrow  {n1phiEqual1} CO_{2} + {n2phiEqual1} CO + {n3phiEqual1} H_{2}O + {n4phiEqual1} N_{2}$' #→

            if EGR_VALUE > 1:
                
                if n1phiEqual1 <= 0 and FuelOption1 != 21:
                    finalEq =  f'$+ {x}(O_{2} + 3.76 N_{2}) \longrightarrow {round(n2phiEqual1 * EGR_VALUE,2)} CO + {round(n3phiEqual1* EGR_VALUE,2)} H_{2}O + {round(n4phiEqual1* EGR_VALUE,2)} N_{2}$' #→
                
                elif FuelOption1 == 21 and FuelOption2 == 22 and FuelOption3 == 22:
                    finalEq =  f'$+ {x}(O_{2} + 3.76 N_{2}) \longrightarrow {round(n3phiEqual1* EGR_VALUE,2)} H_{2}O + {round(n4phiEqual1* EGR_VALUE,2)} N_{2}$' #→
                
                elif FuelOption1 == 21 and FuelOption2 == 21:
                    finalEq =  f'$+ {x}(O_{2} + 3.76 N_{2}) \longrightarrow {round(n3phiEqual1* EGR_VALUE,2)} H_{2}O + {round(n4phiEqual1* EGR_VALUE,2)} N_{2}$' #→
                elif FuelOption1 == 21 and FuelOption2 == 21 and FuelOption3 == 21:
                    finalEq =  f'$+ {x}(O_{2} + 3.76 N_{2}) \longrightarrow {round(n3phiEqual1* EGR_VALUE,2)} H_{2}O + {round(n4phiEqual1* EGR_VALUE,2)} N_{2}$' #→
                elif FuelOption1 != 22 and FuelOption2 != 22 and FuelOption3 != 22 and n1phiEqual1 <= 0 :
                    finalEq =  f'$+ {x}(O_{2} + 3.76 N_{2}) \longrightarrow  {round(n2phiEqual1* EGR_VALUE,2)} CO + {round(n3phiEqual1* EGR_VALUE,2)} H_{2}O + {round(n4phiEqual1* EGR_VALUE,2)} N_{2}$' #→
                else:
                    finalEq =  f'$+ {x}(O_{2} + 3.76 N_{2}) \longrightarrow  {round(n1phiEqual1 * EGR_VALUE,2)} CO_{2} + {round(n2phiEqual1* EGR_VALUE,2)} CO + {round(n3phiEqual1* EGR_VALUE,2)} H_{2}O + {round(n4phiEqual1* EGR_VALUE,2)} N_{2}$' #→
                    
# ################################## LEAN  ########################## #
        elif phiValue < 1:
            n1_phiMinus_1 = round(totalCarbonCheck * phiValue, 2)

            n2_phiMinus_1 = round(totalHydrogen * phiValue, 2 )

            x_phiMinus_1 = round(totalOxygen,2)
            
            n3_phiMinus_1 =  round(x * (1 - phiValue),2) 

            n4_phiMinus_1 = round(3.76 * totalOxygen,2)

            finalEq =  f'$+ {x_phiMinus_1}(O_{2} + 3.76 N_{2}) \longrightarrow  {n1_phiMinus_1} CO_{2} + {n2_phiMinus_1} H_{2}O + {n3_phiMinus_1} O_{2} + {n4_phiMinus_1} N_{2}$' #→
            if n1_phiMinus_1 <= 0:
                finalEq =  f'$+ {x_phiMinus_1}(O_{2} + 3.76 N_{2}) \longrightarrow {n2_phiMinus_1} H_{2}O + {n3_phiMinus_1} O_{2} + {n4_phiMinus_1} N_{2}$' #→

            if EGR_VALUE > 1:
                n1_phiMinus_1_EGR = round(EGR_VALUE * totalCarbonCheck * phiValue, 2)

                n2_phiMinus_1_EGR = round(EGR_VALUE * totalHydrogen * phiValue, 2 )
                x = round(totalOxygen,2)

                n3_phiMinus_1_EGR = round(x * (1 - phiValue) * EGR_VALUE,2)
                
            
                n4_phiMinus_1_EGR = round(EGR_VALUE * 3.76 * totalOxygen,2)


                finalEq =  f'$+ {x}(O_{2} + 3.76 N_{2}) \longrightarrow  {n1_phiMinus_1_EGR} CO_{2} + {n2_phiMinus_1_EGR} H_{2}O + {n3_phiMinus_1_EGR} O_{2} + {n4_phiMinus_1_EGR} N_{2}$' #→
                if n1 <= 0:
                    finalEq =  f'$+ {x}(O_{2} + 3.76 N_{2}) \longrightarrow {n2_phiMinus_1_EGR} H_{2}O + {n3_phiMinus_1_EGR} O_{2} + {n4_phiMinus_1_EGR} N_{2}$' #→

        return result + finalEq
    
# ############ # # # # ################## # # # ################## # # # ################## 

#-7 Table results

@app.callback(
    Output('table_results', 'data'),
    [Input('Concentration-1', 'value'),
     Input('Concentration-2', 'value'),
     Input('Concentration-3', 'value'),
     Input('FuelOption-1', 'value'),
     Input('FuelOption-2', 'value'),
     Input('FuelOption-3', 'value'),
     Input('balanceEq', 'children'),
     Input('slider_value_egr', 'data'),
     Input('slider_value_phi', 'data'),
     
    ],
    [State('table_results', 'data')],
)

def update_table_results(Concentration1, Concentration2, Concentration3, 
                         FuelOption1,FuelOption2, FuelOption3, 
                         balanceEq, EGR_VALUE, PHI_VALUE, table_data):

    if balanceEq is not None:
        # airMultiplier = None  # default value
        airMultiplier = 0
        FuelAirRatio = 0
        

        equation_split = balanceEq.split(" \longrightarrow ")
        leftSideEq = equation_split[0]
        righttSideEq = equation_split[1]

        if '(O_2 + 3.76 N_2)' in leftSideEq:
            airMultiplier = get_airMultiplier(leftSideEq)

        totalCarbon = get_Carbon_FullBalance(leftSideEq)        
        totalHydrogen = get_Hydrogen_FullBalance(leftSideEq)
        totalOxygen = get_Oxygen_FullBalance(leftSideEq)
        
        airMass = 2 * OXYGENMASS + RatioOxy_Nitrogen * 2 * NITROGENMASS

        if FuelOption1 != 22 and Concentration1 is not None and FuelOption2 == 22 :
            if Concentration1 < 100:
                airMultiplier = checkWhatFuelis(FuelOption1, Concentration)
        
                FuelAirRatio = round((airMultiplier * airMass) / ((CARBONMASS * totalCarbon) + (HYDROGENMASS * totalHydrogen) + (OXYGENMASS * totalOxygen)),2)  

            elif Concentration1 == 100:
                FuelAirRatio = round((airMultiplier * airMass) / ((CARBONMASS * totalCarbon) + (HYDROGENMASS * totalHydrogen) + (OXYGENMASS * totalOxygen)),2)  

        elif FuelOption1 != 22 and Concentration1 is not None and FuelOption2 != 22 and Concentration2 is not None and FuelOption3 == 22 :
            FuelAirRatio = round((2 * airMultiplier * airMass) / ((CARBONMASS * totalCarbon) + (HYDROGENMASS * totalHydrogen) + (OXYGENMASS * totalOxygen)),2)  


        elif FuelOption1 != 22 and Concentration1 is not None and FuelOption2 != 22 and Concentration2 is not None and FuelOption2 != 22 and Concentration3 is not None:
            FuelAirRatio = round((  3 *  airMultiplier * airMass) / ((CARBONMASS * totalCarbon  ) + (HYDROGENMASS * totalHydrogen) + (OXYGENMASS * totalOxygen)),2)  

    
# ############ # # # # ################## # # # ################## # # # ########################

    # Mass and % of fuels
        eq1_new_00 = leftSideEq 
        equations_new_00 = eq1_new_00.split("$+")
        equations_new = equations_new_00[0].split(" + ")


        if len(equations_new) == 1:
            eq1_new = equations_new[0]
            eq2_new = ""
            eq3_new = ""
        elif len(equations_new) == 2:
            eq1_new = equations_new[0]
            eq2_new = equations_new[1]
            eq3_new= ""
        elif len(equations_new) == 3:
            eq1_new = equations_new[0]
            eq2_new = equations_new[1]
            eq3_new = equations_new[2]

        count_c_eq1 = get_Carbon_FullBalance(eq1_new)
        count_c_eq2 = get_Carbon_FullBalance(eq2_new)
        count_c_eq3 = get_Carbon_FullBalance(eq3_new)

        count_h_eq1 = get_Hydrogen_FullBalance(eq1_new)
        count_h_eq2 = get_Hydrogen_FullBalance(eq2_new)
        count_h_eq3 = get_Hydrogen_FullBalance(eq3_new)

        count_o_eq1 = get_Oxygen_FullBalance(eq1_new)
        count_o_eq2 = get_Oxygen_FullBalance(eq2_new)
        count_o_eq3 = get_Oxygen_FullBalance(eq3_new)

        massFuel1_Kg = round((count_c_eq1 * CARBONMASS) + (HYDROGENMASS *
                        count_h_eq1) + (OXYGENMASS * count_o_eq1 ),2)

        massFuel2_Kg = round((count_c_eq2 * CARBONMASS) + (HYDROGENMASS * 
                      count_h_eq2) + (OXYGENMASS * count_o_eq2 ),2)

        massFuel3_Kg = round((count_c_eq3 * CARBONMASS) + (HYDROGENMASS *
                      count_h_eq3) + (OXYGENMASS * count_o_eq3 ),2)

        percentageMassFuel1_Kg = round((massFuel1_Kg /
                            (massFuel1_Kg + massFuel2_Kg + massFuel3_Kg)) * 100,2)

        percentageMassFuel2_Kg = round((massFuel2_Kg / 
                            (massFuel1_Kg + massFuel2_Kg + massFuel3_Kg)) * 100,2)

        percentageMassFuel3_Kg = round((massFuel3_Kg / 
                            (massFuel1_Kg + massFuel2_Kg + massFuel3_Kg)) * 100,2)
# ############ # # # # ################## # # # ################## # # # ########################
#         #CO2 Kg 
        
        coefficient_CO2 = getCoef_CO2(righttSideEq)
        
        TotalCo2Kg_0 = coefficient_CO2 * (CARBONMASS + (2 * OXYGENMASS))
        TotalCo2Kg = round(TotalCo2Kg_0 / (massFuel1_Kg),2)
        
        if FuelOption1 != 22 and FuelOption2 != 22 and FuelOption3 == 22:
            TotalCo2Kg = round((TotalCo2Kg_0 / (massFuel1_Kg + massFuel2_Kg)) * 2 ,2) 
                
        elif FuelOption1 != 22 and FuelOption2 != 22 and FuelOption3 != 22:
            TotalCo2Kg = round( (TotalCo2Kg_0 / (massFuel1_Kg + massFuel2_Kg + massFuel3_Kg)) * 3 ,2)

# ############ # # # # ################## # # # ################## # # # ########################
#Enthalpy, Calorifc Value, HHV
        LHV1 =0
        HHV1 =0
        LHV2 =0
        HHV2 =0
        LHV3 =0
        HHV3 =0
        h=0
        h2=0
        h3=0
        righttSideEq_egr = righttSideEq
        
        if EGR_VALUE > 1:
            species = righttSideEq_egr.split(' + ')
            for i in range(len(species)):
                if 'CO_2' in species[i]:
                    species[i] = f"{float(species[i].split()[0])/EGR_VALUE:.2f} CO_2"
                elif 'H_2O' in species[i]:
                    species[i] = f"{float(species[i].split()[0])/EGR_VALUE:.2f} H_2O"
                elif 'N_2' in species[i]:
                    species[i] = f"{float(species[i].split()[0])/EGR_VALUE:.2f} N_2"
                elif 'CO' in species[i]:
                    species[i] = f"{float(species[i].split()[0])/EGR_VALUE:.2f} CO"
                elif 'O_2' in species[i]:
                    species[i] = f"{float(species[i].split()[0])/EGR_VALUE:.2f} O_2"
            righttSideEq_egr = ' + '.join(species)

        if FuelOption1 != 22 and FuelOption2 == 22 and FuelOption3 == 22 :
            f = multiply_coefficients(enthalpy_dataMJKmol, righttSideEq_egr)
            g = calculate_enthalpy_products(leftSideEq, inverted_dict)
            h = f + g # entalphy 
            LHV1 = h / massFuel1_Kg

            i = multiply_coefficients(enthalpy_dataMJKmol_Liquid, righttSideEq_egr)
            j = i + g
            HHV1 = j / massFuel1_Kg

        if FuelOption1 != 22 and FuelOption2 !=22 and FuelOption3 == 22:

            f2 = multiply_coefficients(enthalpy_dataMJKmol, righttSideEq_egr)
            g2 = calculate_enthalpy_products(leftSideEq, inverted_dict) / 2
            h2 = (f2 + g2) # entalphy 
            LHV2 =   2 * (h2 / ((massFuel1_Kg) + (massFuel2_Kg)))

            i2 = multiply_coefficients(enthalpy_dataMJKmol_Liquid, righttSideEq_egr)
            j2 = i2 + g2
            HHV2 = 2 * (j2 / (massFuel1_Kg + massFuel2_Kg))

        if FuelOption1 != 22 and FuelOption2 !=22 and FuelOption3 !=22 :

            f3 = multiply_coefficients(enthalpy_dataMJKmol, righttSideEq_egr)
            g3 = calculate_enthalpy_products(leftSideEq, inverted_dict) / 3
            h3 = (f3 + g3) 
            LHV3 =   3 * (h3 / (massFuel1_Kg + massFuel2_Kg + massFuel3_Kg ))

            i3 = multiply_coefficients(enthalpy_dataMJKmol_Liquid, righttSideEq_egr)
            j3 = i3 + g3
            HHV3 = 3 * (j3 / (massFuel1_Kg + massFuel2_Kg + massFuel3_Kg ))
        
        LHV_Last = round(LHV1 + LHV2 + LHV3, 2)
        HHV_Last = round(HHV1 + HHV2 + HHV3, 2)
        enthalpyRelease = round(h + h2 + h3, 2)

# ############ # # # # ################## # # # ################## # # # ########################
#Instantaneous AFR

        AFRatMoment = round(FuelAirRatio * EGR_VALUE * PHI_VALUE, 2)

# ############ # # # # ################## # # # ################## # # # ########################
# # TEMP By guess
        
        trendpoly_co2 = polyfit_CO2()   
        trendpoly_o2 = polyfit_O2()
        trendpoly_n2 = polyfit_N2()     
        trendpoly_h2o = polyfit_H2O()
        trendpoly_co = polyfit_CO() 


        coeficiente_co2 = coeficiente_co2_func(righttSideEq)
        coeficiente_co = coeficiente_co_func(righttSideEq)
        coeficiente_n2 = coeficiente_n2_func(righttSideEq)
        coeficiente_h2o = coeficiente_h2o_func(righttSideEq)
        coeficiente_o2 = coeficiente_o2_func(righttSideEq)

        Temp = 300 # Kelvin
        FlameTemp = 0
        hf = (coeficiente_co2 * trendpoly_co2(Temp) ) + (coeficiente_o2 * 
                               trendpoly_o2(Temp)) + (coeficiente_h2o * 
                               trendpoly_h2o(Temp)) + (coeficiente_n2 *
                               trendpoly_n2(Temp)) + (coeficiente_co *
                               trendpoly_co(Temp))

        while hf <= enthalpyRelease:
            FlameTemp += 1
            hf = (coeficiente_co2 * trendpoly_co2(FlameTemp) ) + (coeficiente_o2 * 
                trendpoly_o2(FlameTemp)) + (coeficiente_h2o * 
                trendpoly_h2o(FlameTemp)) + (coeficiente_n2 *
                trendpoly_n2(FlameTemp)) + (coeficiente_co * trendpoly_co(FlameTemp))
        
# ############ # # # # ################## # # # ################## # # # ########################


        # Construct the new dataframe to be displayed
        data3 = {'Results': ['Ideal AFR by Mass', 'Instantaneous AFR', 'Mass Per 1 kmol, Fuel Nº 1', 
                             'Mass Per 1 kmol, Fuel Nº 2', 'Mass Per 1 kmol, Fuel Nº 3',
                             'Mass Percentage Fuel Nº 1  ', 'Mass Percentage Fuel Nº 2  ',
                            'Mass Percentage Fuel Nº 3  ', 'Total Co2 Kg', 'enthalpyRelease',
                            'LHV', 'HHV', 'Flame Temperature'],

                '-': [FuelAirRatio, AFRatMoment, massFuel1_Kg, massFuel2_Kg, massFuel3_Kg, 
                      percentageMassFuel1_Kg, percentageMassFuel2_Kg, percentageMassFuel3_Kg,
                     TotalCo2Kg, enthalpyRelease, LHV_Last, HHV_Last, FlameTemp],

                'Units': ['', '', 'Kg', 'Kg', 'Kg', '%', '%', '%', 'Kg', 
                          'MJ/kmol', 'MJ/kmol', 'MJ/kmol', 'Kelvin']
               }

        results = pd.DataFrame(data3)

        # Convert the dataframe to dict format for use in DataTable
        data = results.to_dict('records')
        return data

#-1
@app.callback(
Output('outputCalc', 'children'),
    Input('submit-buttonCalc', 'n_clicks'),
    
        [      State('nCylindersVar', 'value'),
               State('conRodLengVar', 'value'),
               State('strokeVar', 'value'),
               State('boreVar', 'value'),
               State('compVar', 'value'),
               State('t_ambVar', 'value'),
               State('throttleVar', 'value'),
               State('RPMVar', 'value'),
               State('igDelayVar', 'value'),
               State('igdegVar', 'value'),
               State('burnDVar', 'value'),
               State('table_results', 'data'),
                ]
)
def check_inputs(n_clicks,n_cylinders, con_rod_length, stroke, bore, compression_ratio, 
                 ambient_temperature, throttle, rpm, ignition_delay, 
                 ignition_degrees, burn_duration,data): 
    if n_clicks is None:
        n_clicks = 0
        
    elif n_clicks > 0:
    
        if not all([n_cylinders, con_rod_length, stroke, bore, compression_ratio, ambient_temperature, 
                   throttle, rpm, ignition_delay, ignition_degrees, burn_duration]):
            return 'Please fill all the inputs.'
        else :
            if data is not None:
                AFR_FromTable = data[1]['-']
                LHV_FromTable = data[10]['-'] * pow(10,6)
                
                EngineCalc(n_cylinders, con_rod_length, stroke, bore, compression_ratio, 
                   ambient_temperature, throttle, rpm, 
                   ignition_delay, ignition_degrees, burn_duration, AFR_FromTable, 
                   LHV_FromTable, df)
                n_clicks = 0

# ############ # # # # ################## # # # ################## # # # ##################
#-2
@app.callback(
Output('graph_engine','children'), 
[Input('dropdown1','value'), Input('submit-buttonCalc', 'n_clicks'),
])
def dropdown_menu(value, n_clicks):
    if n_clicks is None:
        return dash.no_update
    elif n_clicks > 0:
        titlex = 'Crank Angle'
        if value=='1':
            xx= df['Crank Angle']
            y1= df['Piston Position [mm]']
            col='lightcoral'
            titley = 'Piston Position [mm]'
            
        elif value=='2':
            xx= df['Crank Angle']
            y1= df['Piston Velocity [m/s]']
            col='skyblue'
            titley = 'Piston Velocity [m/s]'
            
        elif value=='3':
            xx= df['Crank Angle']
            y1= df['Piston Acc [m/s/s]']
            col='skyblue'
            titley = 'Piston Acc [m/s/s]'
            
        elif value=='4':
            xx= df['Crank Angle']
            y1= df['Volume [CC]']
            col='skyblue'
            titley = 'Volume [CC]'
            
        elif value=='5':
            xx= df['Crank Angle']
            y1= df['Combustion [bar]']
            col='skyblue'
            titley = 'Combustion [bar]'
            
        elif value=='6':
            xx= df['Crank Angle']
            y1= df['Temperature [K]']
            col='skyblue'
            titley = 'Temperature [K]'
            
        elif value=='7':
            xx= df['Crank Angle']
            y1= df['Work Done [J]']
            col='skyblue'
            titley = 'Work Done [J]'
            
        elif value=='8':
            xx= df['Crank Angle']
            y1= df['MFB']
            col='skyblue'  
            titley = 'MFB'          
            
        elif value=='9':
            xx= df['Volume [CC]']
            y1= df['Combustion [bar]']
            col='skyblue' 
            titley = 'Combustion [bar]'            
        elif value=='10':
            return {}
        fig = go.Figure(data=[go.Scatter(x=xx, y=y1, marker_color= 'skyblue')])
        fig.update_layout(title='', 
                          plot_bgcolor='white', paper_bgcolor='white', xaxis_tickangle=-17,
                         xaxis=dict(
#                         type='line',
                        title= titlex,
                        showgrid=True,
                        showline=True,
                        color='black',
                        linewidth=2,
                        zeroline=True,

                        ),
                        yaxis=dict(
                        title= titley,
                        showgrid= True,
                        showline= True,
                        color='black',
                        linewidth=2,
                        zeroline=True
                        ),
#                         margin={'l': 50, 'b': 30, 't': 30, 'r': 30},

                        hovermode='closest',
                         
                         )
        graph = dcc.Graph(figure=fig)
        

        return graph
    

# 5. Callback function to show the results WorkDone, Power, Qr, BHP, Engine Capacity
@app.callback(
    Output('results_engine', 'children'),
    
     Input('submit-buttonCalc', 'n_clicks'),
    )
def update_result(n_clicks):
        if n_clicks is None:
            return dash.no_update
        elif n_clicks > 0:
            
            TotalWorkDone = dataResults['Results'][0]
            PowerKw = dataResults['Results'][1]
            BHP = dataResults['Results'][2]
            EngineCapacity = dataResults['Results'][3]
            Qr = dataResults['Results'][4]
            # update the Results column in the dataResults dataframe

            results_table = generate_table(pd.DataFrame(columns=['Results', '  '], 
                                       data=[['{}'.format(dataResults['Engine Cycle Simulation'][i]), '{}'.format(dataResults['Results'][i])] 
                                             for i in range(len(dataResults['Engine Cycle Simulation']))]))

            return results_table
        

@app.callback(
Output('graph_nOX','children'), 
[Input('submit-buttonNox', 'n_clicks'), 
],
 [State('table_results', 'data'),
 State('strokeVar', 'value'), Input('nCylindersVar', 'value'),
 State('boreVar', 'value'), Input('throttleVar', 'value'), 
 State('t_ambVar', 'value'), Input('slider_value_phi', 'data'),
 State('balanceEq', 'children'), Input('RPMVar', 'value'),]

)
def graphNoX(n_clicks, data, Stroke, Cylinders, BORE_IN, Throttle_IN, InitialTemp, PHI_VALUE, balanceEq,
            RPM_Func):
    new_df = {}
    if n_clicks is None:
        return dash.no_update
    elif n_clicks > 0 and PHI_VALUE < 1:

        flame_Temp = data[-1]['-'] # ok
        
        BORE_IN = BORE_IN / 1000 # m
        Stroke = Stroke / 1000 # m
        Vts = Cylinders * (math.pi / 4) * ((BORE_IN) ** 2) * (Stroke)  # Total swept volume m^3
        Vsv = Vts / Cylinders  # Swept volume or cylinder volume
        volume_cylinder = Vsv

        InitialTemp = InitialTemp + 273 # Input('t_ambVar', 'value'),
        atmospheric_pressure = 1 #bar

        new_df = RateChangePressure(df)
        comb_chamber_volume = new_df['Volume [CC]'][0] * pow(10,-6)


        x = nOX(flame_Temp, volume_cylinder,Throttle_IN, InitialTemp, PHI_VALUE, balanceEq,
           comb_chamber_volume, RPM_Func, atmospheric_pressure, new_df)

        
        titlex = 'Temperature'
        xx = new_df['Temp [K]']
        y1_1 = new_df['NoX [kmols/°]'].round(12)
        y1_2 = new_df['Crank Angle']
        col = 'skyblue' 
        titley = 'NoX [kmols/°] & Crank Angle'            

        fig = go.Figure()

        # Add a line trace for y1_1
        fig.add_trace(go.Scatter(
            x=xx,
            y=y1_1,
            mode='lines',
            name='NoX [kmols/°]',
            line=dict(color=col)
        ))

        # Add a scatter trace for y1_2
        fig.add_trace(go.Scatter(
            x=xx,
            y=y1_2,
            mode='markers',
            name='Crank Angle',
            yaxis='y2',
            marker=dict(color='red', size=3)
        ))

        # Define the layout of the figure
        fig.update_layout(
            title={
                    'text': 'Nox',
                    'y': 0.95,
                    'x': 0.5,
                    'xanchor': 'center',
                    'yanchor': 'top',
                    'font': dict(size=24),
                },
            plot_bgcolor='white',
            paper_bgcolor='white',
            xaxis_tickangle=-17,
            xaxis=dict(
                title=titlex,
                showgrid=True,
                showline=True,
                color='black',
                linewidth=2,
                zeroline=True
            ),
            yaxis=dict(
                title='NoX [kmols/°]',
                showgrid=True,
                showline=True,
                color='black',
                linewidth=2,
                zeroline=True,
                tickformat='.0e',
                tickmode='auto',
                ticklen=5,
                tickfont=dict(size=12),
                exponentformat='e',
                showexponent='all'
            ),
            yaxis2=dict(
                title='Crank Angle',
                overlaying='y',
                side='right',
                showgrid=False
            ),
#             margin=dict(l=50, b=30, t=30, r=30),
            hovermode='closest'
        )

        graph = dcc.Graph(figure=fig)
        n_clicks = 0

        return graph
    
    

In [11]:
####################################  LINK TO OPEN THE SERVER #############################
app.title = 'Vinnie Dreher - NoX'
app.run_server(debug=True)

# visit http://127.0.0.1:8050/ in your web browser.


Dash app running on http://127.0.0.1:8050/
